In [1]:
from pathlib import Path
import zarr
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from loguru import logger
import h5py
from typing import Union

Array = Union[h5py.Dataset, zarr.Array, np.ndarray]

patch_size = 224

In [2]:
# zarr
feature_folder = Path("/data/data/shiprec/TCGA-BRCA-DX")
slide_folder = feature_folder / "TCGA-4H-AAAK-01Z-00-DX1.ABF1B042-1970-4E28-8671-43AAD393D2F9/"

coords = zarr.open(str(slide_folder / "coords.zarr"), mode="r")
norm_patches = zarr.open(str(slide_folder / "normalized_patches.zarr"), mode="r")
patches = zarr.open(str(slide_folder / "patches.zarr"), mode="r")

In [3]:
# # h5py

# feature_folder = Path("/mnt/bulk/gwoelflein/georg-tcga-slides/shiprec/TCGA-BRCA-DX")
# slide = "TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4BD-C6B76294C291"

# # features_file = feature_folder / "features" / f"{slide}.h5"
# patches_file = feature_folder / "images" / f"{slide}.h5"

# with h5py.File(patches_file, "r") as f:
#     norm_patches = f["normalized_patches"]
#     patches = f["patches"]
#     coords = f["coords"]

In [4]:
def reconstruct_image(patches: Array, coords: Array) -> np.ndarray:
    coords = coords[:]
    patches = patches[:]

    img = np.zeros((*(coords.max(0) + patch_size)[::-1], 3), dtype=np.uint8)
    for (x, y), patch in tqdm(zip(coords, patches), total=coords.shape[0], desc="Reconstructing image"):
        img[y : y + patch_size, x : x + patch_size] = patch
    logger.debug(f"Reconstructed image from {patches.shape[0]} patches as shape {img.shape}")
    return img

In [6]:
cv2.imwrite("/app/patches.png", cv2.cvtColor(reconstruct_image(patches, coords), cv2.COLOR_RGB2BGR))
cv2.imwrite("/app/norm_patches.png", cv2.cvtColor(reconstruct_image(norm_patches, coords), cv2.COLOR_RGB2BGR))

Reconstructing image: 100%|██████████| 2643/2643 [00:00<00:00, 6961.89it/s]
2023-09-11 11:44:00.229 | DEBUG    | __main__:reconstruct_image:8 - Reconstructed image from 2643 patches as shape (16576, 15456, 3)
Reconstructing image: 100%|██████████| 2643/2643 [00:00<00:00, 6990.39it/s]
2023-09-11 11:44:14.590 | DEBUG    | __main__:reconstruct_image:8 - Reconstructed image from 2643 patches as shape (16576, 15456, 3)


True